In [28]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd
import community
from collections import Counter
from webweb import Web # pip install webweb

In [29]:
df = pd.read_pickle("./movie_scripts.pkl")

In [30]:
# Add one between characters that are in the same scene

scene_movie_interactions = dict()

scene_interactions = dict()
for movie in df.movie_nr.unique():
    scene_interactions = dict()
    df_store = df[df.movie_nr == movie].groupby(['scene_id'])['character'].unique()
    for scene in df_store:
        for idx in range(len(scene)-1):
            for idx2 in range(idx+1,len(scene)):
                if (scene[idx],scene[idx2]) in scene_interactions:
                    scene_interactions[(scene[idx],scene[idx2])] += 1
                else:
                    if (scene[idx2],scene[idx]) in scene_interactions:
                        scene_interactions[(scene[idx2],scene[idx])] += 1
                    else:
                        scene_interactions[(scene[idx],scene[idx2])] = 1
    scene_movie_interactions[movie] = scene_interactions

In [31]:
scene_movie_interactions

{1: {('Albus Dumbledore', 'Mcgonagall'): 2,
  ('Albus Dumbledore', 'Hagrid'): 2,
  ('Albus Dumbledore', 'Petunia'): 1,
  ('Albus Dumbledore', 'Dudley'): 1,
  ('Albus Dumbledore', 'Vernon'): 1,
  ('Albus Dumbledore', 'Harry Potter'): 6,
  ('Mcgonagall', 'Hagrid'): 2,
  ('Mcgonagall', 'Petunia'): 1,
  ('Mcgonagall', 'Dudley'): 1,
  ('Mcgonagall', 'Vernon'): 1,
  ('Mcgonagall', 'Harry Potter'): 9,
  ('Hagrid', 'Petunia'): 2,
  ('Hagrid', 'Dudley'): 2,
  ('Hagrid', 'Vernon'): 2,
  ('Hagrid', 'Harry Potter'): 16,
  ('Petunia', 'Dudley'): 3,
  ('Petunia', 'Vernon'): 6,
  ('Petunia', 'Harry Potter'): 4,
  ('Dudley', 'Vernon'): 5,
  ('Dudley', 'Harry Potter'): 5,
  ('Vernon', 'Harry Potter'): 6,
  ('Dudley', 'Snake'): 1,
  ('Vernon', 'Snake'): 1,
  ('Harry Potter', 'Snake'): 1,
  ('Snake', 'Petunia'): 1,
  ('Harry Potter', 'All'): 3,
  ('Vernon', 'All'): 1,
  ('Hagrid', 'All'): 2,
  ('Petunia', 'All'): 1,
  ('All', 'Dudley'): 1,
  ('Harry Potter', 'Tom'): 1,
  ('Harry Potter', 'Witch'): 1,
  (

In [32]:
for x in scene_movie_interactions:
    # Create an undirected weighted graph
    G = nx.Graph()
    current = scene_movie_interactions[x]
    for edge in current:
        edge_1 = edge[0]
        edge_2 = edge[1]
        weight = current[edge]
        G.add_edges_from([(edge_1, edge_2, {'weight': weight})])
    G_GCC = G.subgraph(max(nx.connected_components(G), key=len))
    
    partition = community.best_partition(G_GCC, random_state = 0)
    # Create a dictionary of node name and community number
    communities = {}
    for node in G_GCC.nodes:
        communities[node] = partition[node]

    # Get all the community values in a single variable for counting
    lovain_communities = communities.values()

    # Count and sort the communities
    communities_counter = Counter(lovain_communities).most_common(max(lovain_communities))

    # Add the community numbers to an attribute within the graph
    nx.set_node_attributes(G_GCC, communities, "community")
    
    with open(f'./data/networks_pickled/movie_{x}.pkl', 'wb') as f:
        pickle.dump(G_GCC, f)

In [33]:
# All the movie titles in correct order
movie_titles = ["Harry Potter and the Philosopher's Stone","Harry Potter and the Chamber of Secrets",
              "Harry Potter and the Prisoner of Azkaban","Harry Potter and the Goblet of Fire",
              "Harry Potter and the Order of the Phoenix","Harry Potter and the Half-Blood Prince",
              "Harry Potter and the Deathly Hallows – Part 1","Harry Potter and the Deathly Hallows – Part 2"]
movie_titles_short = [movie_titles[i].split("Harry Potter and the ", 1)[1] for i in range(len(movie_titles))]

In [34]:
new_title = []
for x in movie_titles_short:
    new_title.append("".join(x.split()))
new_title

["Philosopher'sStone",
 'ChamberofSecrets',
 'PrisonerofAzkaban',
 'GobletofFire',
 'OrderofthePhoenix',
 'Half-BloodPrince',
 'DeathlyHallows–Part1',
 'DeathlyHallows–Part2']

In [35]:
network_list = []
for movie in np.arange(1,9):
    with open(f'./data/networks_pickled/movie_{movie}.pkl', 'rb') as f:
        G = pickle.load(f)
    network_list.append(G)

In [36]:
# # Building another copy of the network, to leave out a few attributes that don't plot very well

# G_comms = nx.DiGraph()

# # Looping through other network and adding to the network
# for node in G.nodes:             
#     G_comms.add_node(node, # node name
#                 book_nr = str(G_viz.nodes[node]['book_nr']), 
#                 house = G_viz.nodes[node]['house'], 
#                 sentiment = node_sentiment[node],
#                 degree_centrality = G.nodes[node]['degree_centrality'],
#                 in_degree_centrality = G.nodes[node]['in_degree_centrality'],     
#                 out_degree_centrality = G.nodes[node]['out_degree_centrality'],     
#                 blood = G_viz.nodes[node]['blood'],
#                 gender = G_viz.nodes[node]['gender'],
#                 species = "Human" if G_viz.nodes[node]['species'] == "Human" else "Nonhuman",
#                 community = G_viz.nodes[node]['community'])

# # Adding links
# for node in G_comms.nodes:
#     for link in G_viz.nodes[node]['links']:
#         if link in G_comms: # If other entity is in the network
#             G_comms.add_edge(node, link)
            

# print("Added {} nodes. Size of network: {}".format(len(df_wiki), len(G)))  
# print("{} edges in network.".format(len(G.edges)))


In [37]:
# Declaring webweb instance of the networks

web = Web(title="Philosopher's Stone", nx_G=network_list[0])

# movie1 = network_list[0]
# movie2 = network_list[1]
# movie3 = network_list[2]
# movie4 = network_list[3]
# movie5 = network_list[4]
# movie6 = network_list[5]
# movie7 = network_list[6]
# movie8 = network_list[7]

["Philosopher'sStone",
 'ChamberofSecrets',
 'PrisonerofAzkaban',
 'GobletofFire',
 'OrderofthePhoenix',
 'Half-BloodPrince',
 'DeathlyHallows–Part1',
 'DeathlyHallows–Part2']


web.networks.ChamberofSecrets(nx_G=network_list[1])
web.networks.PrisonerofAzkaban(nx_G=network_list[2])
web.networks.GobletofFire(nx_G=network_list[3])
web.networks.OrderofthePhoenix(nx_G=network_list[4])
web.networks.Half_BloodPrince(nx_G=network_list[5])
web.networks.DeathlyHallows_PartI(nx_G=network_list[6])
web.networks.DeathlyHallows_PartII(nx_G=network_list[7])

In [38]:
# Configuration

web.display.colorBy = 'community'
web.display.sizeBy = 'degree'
web.display.gravity = .1
web.display.charge = 100
web.display.radius = 6
web.display.linkLength = 20
web.display.linkStrength = 0.1
web.display.colorPalette = 'RdYlGn'
web.display.scaleLinkOpacity = True
web.display.scaleLinkWidth = False
web.display.nameToMatch = 'Harry'

In [39]:
# Testing
web.show()

In [40]:
# Saving 
web.save('./data/networks_pickled/script_webweb.html')